In [ ]:
import satimagets as sits
import pandas as pd
import numpy as np
import re
import gdal
import matplotlib.pyplot as plt

In [ ]:
S2_PATH = "Y:\\Harmonisation_PS_S2\\m3sat_s2output\\"#"D:\\M3sat_S2output\\"
PS_PATH = "Y:\PS\Mosaic\PS_5m\\"
PS_OLD = "Y:\\Harmonisation_PS_S2\\output_S2_spatial\\"
PS_PATHre = "Y:\\\\Harmonisation_PS_S2\\\\m3sat_s2output\\\\"
PS_OLDre = "Y:\\\\Harmonisation_PS_S2\\\\output_S2_spatial\\\\"
locations = ["Izola", "Radenci", "Jesenice", "Kranj"]
DF_PATH = "C:\\Users\\mracic\\Univerza v Ljubljani\\Fetai, Bujar - satimagets-master\\"
dfs = pd.read_excel(DF_PATH+"Harmonisation_PS_S2.xlsx")

In [ ]:
# fix the path to S2 file
dfs["S2_path"] = dfs["S2_path"].apply(lambda x: re.sub(PS_OLDre,PS_PATHre,x))
# Add paths to masks
dfs["S2_mask_path"] = dfs["S2_path"].apply(lambda x: re.sub("m_S2","m_mask_S2",x))
dfs["PS_udm_path"] = dfs["PS_Path"].apply(lambda x: re.sub("analytic","udm",x))

In [ ]:
def open_im(im_file):
    try:
        return gdal.Open(im_file)
    except:
        print('Could not open:', im_file)
        sys.exit(1)

def vec_bin_array(arr, m=8):
    """
    Arguments: 
    arr: Numpy array of positive integers
    m: Number of bits of each integer to retain

    Returns a copy of arr with every element replaced with a bit vector.
    Bits encoded as int8's.
    """
    to_str_func = np.vectorize(lambda x: np.binary_repr(x).zfill(m))
    strs = to_str_func(arr)
    ret = np.zeros(list(arr.shape) + [m], dtype=np.int8)
    for bit_ix in range(0, m):
        fetch_bit_func = np.vectorize(lambda x: x[bit_ix] == '1')
        ret[...,bit_ix] = fetch_bit_func(strs).astype("int8")

    return ret

In [ ]:
for loc_name in locations:
    print(loc_name)
    sub_dfs = dfs[dfs["Location"]==loc_name]
    file_1_src = open_im(sub_dfs.iloc[0]["PS_Path"])
    file_2_src = open_im(sub_dfs.iloc[0]["S2_path"])
    # Find overlap
    im_bb, im_res, im_srs = sits.image_raster_overlap(file_1_src, file_2_src, 0)
    # Set resolution
    im_res = 100
    for index, row in sub_dfs.iterrows():
        file_1 = row["PS_Path"]
        file_1_mask = row["PS_udm_path"]
        file_2 = row["S2_path"]
        file_2_mask = row["S2_mask_path"]
        # Load images and masks
        file_1_src = open_im(file_1)
        file_2_src = open_im(file_2)
        file_1_mask = open_im(file_1_mask)
        file_2_mask = open_im(file_2_mask)
        # Resample images and masks
        im_1 = sits.image_raster_resample(file_1_src, im_bb, im_res, im_srs, 0)
        im_2 = sits.image_raster_resample(file_2_src, im_bb, im_res, im_srs, 0)
        im_1_m = sits.image_raster_resample(file_1_mask, im_bb, im_res, im_srs)
        im_2_m = sits.image_raster_resample(file_2_mask, im_bb, im_res, im_srs)
        # Merge masks
        mask = sits.mask_data(im_2_m, im_1_m, 0)
        mask = sits.mask_data(mask, im_2_m, 100)        
        # Mask images
        im_1 = sits.mask_data(im_1, mask, 100)
        im_2 = sits.mask_data(im_2, mask, 100)        
        
        sits.image_scatterplot(im_1[[1,2,3],:,:]/10000, im_2[[1, 2, 3],:,:], im1_in_name="Planet", im2_in_name="Sentinel", sample=1000)
        #sits.image_scatterplot(im_1_ndvi, im_2_ndvi, im1_in_name="Planet", im2_in_name="Sentinel")
        break
    break